In [1]:
import tensorflow as tf
import pandas as pd
import sagemaker
import numpy as np
tf.__version__

#Reading in data in folder
df = pd.read_csv('disneyData.csv')
df.head()

,RideId,ParkId,CreatedTimeStamp,PresentWaitTime,PresentStatus,PresentIsActive,OneWaitTime,OneStatus,OneIsActive,TwoWaitTime,...,TwentyIsActive,TwentyOneWaitTime,TwentyOneStatus,TwentyOneIsActive,TwentyTwoWaitTime,TwentyTwoStatus,TwentyTwoIsActive,TwentyThreeWaitTime,TwentyThreeStatus,TwentyThreeIsActive
0,41,4,2020-07-14 00:10:00,0,Closed,0,0,Closed,0,0,...,0,0,Closed,0,0,Closed,0,0,Closed,0
1,50,4,2020-07-14 00:10:00,0,Closed,0,0,Closed,0,0,...,0,0,Closed,0,0,Closed,0,0,Closed,0
2,51,4,2020-07-14 00:10:00,0,Closed,0,0,Closed,0,0,...,0,0,Closed,0,0,Closed,0,0,Closed,0
3,54,4,2020-07-14 00:10:00,0,Closed,0,0,Closed,0,0,...,0,0,Closed,0,0,Closed,0,0,Closed,0
4,55,4,2020-07-14 00:10:00,0,Closed,0,0,Closed,0,0,...,0,0,Closed,0,0,Closed,0,0,Closed,0


In [2]:
numRows = df.shape[0]

xPoints = []
yPoints = []
for i in range(0, numRows, 38):
    timeFrame = df.loc[i:i+37]
    times = list(timeFrame[timeFrame.columns[range(3, 74, 3)]].to_numpy().reshape(1, 38*24)[0])
    statuses = list(timeFrame[timeFrame.columns[range(5, 75, 3)]].to_numpy().reshape(1, 38*24)[0])
    row = times + statuses
    row = np.array(row).reshape(76, 24)
    xPoints.append(row[:,0:12])
    yPoints.append(row[:,12:24])

xPoints = np.array(xPoints)
yPoints = np.array(yPoints)
p = np.random.permutation(xPoints.shape[0])
xPoints = xPoints[p]
yPoints = yPoints[p]

In [3]:
#sample reshape
#test = np.array(dataPoints).reshape(2, 1104)[0]
#for i in range(0, 46):
#    print(test.reshape(46, 24)[i])


In [6]:
size = xPoints.shape[0]
maxTrain = int(size * 0.8)
print(maxTrain)

trainData = {
    "x": xPoints[0:maxTrain],
    "y": yPoints[0:maxTrain]
}
testData = {
    "x": xPoints[maxTrain:],
    "y": yPoints[maxTrain:]
}

import pickle
with open('disney_train.pkl', 'wb') as f:
    pickle.dump(trainData, f)
with open('disney_test.pkl', 'wb') as f:
    pickle.dump(testData, f)


90611


In [7]:
import boto3
sagemaker_session = sagemaker.Session()

#Uploading data to S3 bucket titled "tf-iris-data"
prefix = "disney-data"
training_input_path = sagemaker_session.upload_data('disney_train.pkl', bucket='sagemaker-park-data', key_prefix=prefix)
training_input_path
testing_input_path = sagemaker_session.upload_data('disney_test.pkl', bucket='sagemaker-park-data', key_prefix=prefix)


In [8]:
print(training_input_path)
print(testing_input_path)

s3://sagemaker-park-data/disney-data/disney_train.pkl
s3://sagemaker-park-data/disney-data/disney_test.pkl
